# Part I. ETL Pipeline for Pre-Processing the Files

### Creating ETL pipeline to model data from csv files into NoSQL data base - Apache Cassandra

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
full_data_rows_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    for f in file_path_list:
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            csvreader = csv.reader(csvfile) 
            next(csvreader)
            for line in csvreader:
                full_data_rows_list.append(line)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6987


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Creating queries to ask the following three questions of the data

### QUERY 1

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
def delete_table(x):
    query = "drop table if exists" +" " +x
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

delete_table("music_events")                    

query = "CREATE TABLE IF NOT EXISTS music_events "
query = query + "(session_id int,\
                iteminsession int,\
                artist_name text,\
                song_title text,\
                song_length float,\
                primary key (session_id, iteminsession, artist_name, song_title))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_events(session_id,\
                                            iteminsession,\
                                            artist_name,\
                                            song_title,\
                                            song_length)"
        
        query = query + "values(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])) )

#### Performing a SELECT to fetch data as requested in query 1

In [10]:
query = "select artist_name, song_title, song_length from music_events\
                where session_id = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,"---" , row.song_title, "---" ,row.song_length)

Faithless --- Music Matters (Mark Knight Dub) --- 495.30731201171875


### QUERY 2

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
delete_table("user_table")                    

query = "CREATE TABLE IF NOT EXISTS user_table "
query = query + "(user_id int,\
                    session_id int,\
                    iteminsession int,\
                    first_name text, \
                    last_name text,\
                    artist_name text, \
                    song_title text, \
                    primary key((user_id, session_id), iteminsession, first_name, last_name))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into user_table (user_id,\
                                        session_id,\
                                        iteminsession,\
                                        first_name,\
                                        last_name,\
                                        artist_name,\
                                        song_title)"
    
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### Performing a SELECT to fetch data as requested in query 2

In [13]:
query = "select artist_name, song_title, first_name, last_name from user_table\
        where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name,"---" ,row.song_title, "---" ,row.first_name,"---" , row.last_name)                    

Down To The Bone --- Keep On Keepin' On --- Sylvie --- Cruz
Three Drives --- Greece 2000 --- Sylvie --- Cruz
Sebastien Tellier --- Kilometer --- Sylvie --- Cruz
Lonnie Gordon --- Catch You Baby (Steve Pitron & Max Sanna Radio Edit) --- Sylvie --- Cruz


### QUERY 3

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
delete_table("song_table")                    

query = "CREATE TABLE IF NOT EXISTS song_table "
query = query + "(song_title text,\
                    first_name text,\
                    last_name text,\
                    user_id int, \
                    primary key(song_title, first_name, last_name, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_table(song_title, first_name, last_name, user_id)"
        query = query + "values(%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])  ))

#### Performing a SELECT to fetch data as requested in query 3

In [17]:
query = "select first_name, last_name from song_table \
        where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,"---" , row.last_name)   

Jacqueline --- Lynch
Sara --- Johnson
Tegan --- Levine


### Drop the tables before closing out the sessions

In [18]:
delete_table("music_events")
delete_table("user_table")
delete_table("song_table")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()